# Transform

In [1]:
import pathlib
import sys
import pickle

from typing import List

import numpy as np
import pandas as pd

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(str(src_dir))
# import my class code from the source
# %aimport src-dir.filename
from data.data_process import transform_ts_to_supervised

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PROJECT_DIR = pathlib.Path.cwd().resolve().parent
PREPPED_DATA_DIR = PROJECT_DIR / 'data' / '05-prepped-for-transform'
TRANS_DATA_DIR = PROJECT_DIR / 'data' / '06-transformed'
CONSOL_DATA_DIR = PROJECT_DIR / 'data' / '07-consolidated'

In [2]:
nations = ['united-kingdom']
leagues = ['english-premier-league']
seasons = ['2000-2001', '2001-2002', '2002-2003', '2003-2004',
           '2004-2005', '2005-2006', '2006-2007', '2007-2008',
           '2008-2009', '2009-2010', '2010-2011', '2011-2012',
           '2012-2013', '2013-2014', '2014-2015', '2015-2016',
           '2016-2017', '2017-2018']

In [3]:
def form_fdcu_fps(PREPPED_DATA_DIR, nation, league, seasons):
    """
    Accepts directory names to enable reach into raw data directory
    Returns full filepaths of the data files
    This is different - different signature no source - to other versions - needs to be standardized
    """
    fps = []
    fdcu_specifics = ['football-data-co-uk', 'season-data']
    for season in seasons:
        fn = season + '.csv'
        fp = PREPPED_DATA_DIR / fn
        #if fp.is_file():
        if fp.exists():
            fps.append(fp)
    return fps

#fps = form_fdcu_fps(MERGED_DATA_DIR, nations[0], leagues[0], seasons)

In [4]:
all_fps = []
for nation, league in zip(nations, leagues):
    fps = form_fdcu_fps(PREPPED_DATA_DIR, nation, league, seasons)
    all_fps.extend(fps)

print(len(all_fps))

11


In [5]:
season_dfs = []
for fp in all_fps:
    season_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
    season_dfs.append(season_df)
season_dfs[0].head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean
0,united-kingdom,english-premier-league,2007-2008,2007-08-11,aston-villa,liverpool,1.0,2.0,4.0,18.0,0.0,10.0,6.0,0.0,4.0,2.0,11.0,0.0,17.0,7.0,1.0,2.0,3.67,3.23,1.95,4.00,3.25,1.90,3.95,3.37,2.02,4.00,3.40,2.10,awin,loss,win,1.0,2.0,2.0,1.0,0.248863,0.282764,0.282764,0.468373,3.23
1,united-kingdom,english-premier-league,2007-2008,2007-08-11,bolton-wanderers,newcastle-united,1.0,3.0,4.0,15.0,0.0,13.0,9.0,0.0,1.0,3.0,16.0,0.0,7.0,5.0,3.0,1.0,2.41,3.18,2.80,2.50,3.20,2.75,2.60,3.10,2.90,2.60,3.25,3.05,awin,loss,win,1.0,3.0,3.0,1.0,0.381887,0.289417,0.289417,0.328696,3.18
2,united-kingdom,english-premier-league,2007-2008,2007-08-11,derby,portsmouth,2.0,2.0,6.0,14.0,0.0,12.0,5.0,1.0,1.0,6.0,17.0,0.0,12.0,6.0,1.0,2.0,2.85,3.19,2.38,2.80,3.25,2.40,3.20,3.20,2.35,3.25,3.30,2.55,draw,draw,draw,2.0,2.0,2.0,2.0,0.323531,0.289048,0.289048,0.387421,3.19
3,united-kingdom,english-premier-league,2007-2008,2007-08-11,everton,wigan-athletic,2.0,1.0,6.0,8.0,0.0,12.0,8.0,1.0,0.0,2.0,13.0,0.0,14.0,4.0,0.0,0.0,1.66,3.45,5.25,1.66,3.40,5.50,1.73,3.50,5.40,1.75,3.65,6.00,hwin,win,loss,2.0,1.0,1.0,2.0,0.556375,0.267705,0.267705,0.175920,3.45
4,united-kingdom,english-premier-league,2007-2008,2007-08-11,middlesbrough,blackburn-rovers,1.0,2.0,13.0,16.0,0.0,10.0,6.0,1.0,3.0,3.0,16.0,0.0,4.0,4.0,0.0,4.0,2.41,3.18,2.81,2.37,3.25,2.87,2.57,3.20,2.95,2.70,3.25,2.95,awin,loss,win,1.0,2.0,2.0,1.0,0.382334,0.289756,0.289756,0.327909,3.18


In [6]:
season_dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 46 columns):
nation                   380 non-null object
league                   380 non-null object
season                   380 non-null object
date                     380 non-null datetime64[ns]
h                        380 non-null object
a                        380 non-null object
h_ftGoals                380 non-null float64
a_ftGoals                380 non-null float64
h_corners                380 non-null float64
h_fouls                  380 non-null float64
h_redCards               380 non-null float64
h_shots                  380 non-null float64
h_shotsOnTarget          380 non-null float64
h_htGoals                380 non-null float64
h_yellowCards            380 non-null float64
a_corners                380 non-null float64
a_fouls                  380 non-null float64
a_redCards               380 non-null float64
a_shots                  380 non-null float64
a_shotsOnTarget    

### Select Columns, Tidy up Column names

In [7]:
season_dfs[0].head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean
0,united-kingdom,english-premier-league,2007-2008,2007-08-11,aston-villa,liverpool,1.0,2.0,4.0,18.0,0.0,10.0,6.0,0.0,4.0,2.0,11.0,0.0,17.0,7.0,1.0,2.0,3.67,3.23,1.95,4.00,3.25,1.90,3.95,3.37,2.02,4.00,3.40,2.10,awin,loss,win,1.0,2.0,2.0,1.0,0.248863,0.282764,0.282764,0.468373,3.23
1,united-kingdom,english-premier-league,2007-2008,2007-08-11,bolton-wanderers,newcastle-united,1.0,3.0,4.0,15.0,0.0,13.0,9.0,0.0,1.0,3.0,16.0,0.0,7.0,5.0,3.0,1.0,2.41,3.18,2.80,2.50,3.20,2.75,2.60,3.10,2.90,2.60,3.25,3.05,awin,loss,win,1.0,3.0,3.0,1.0,0.381887,0.289417,0.289417,0.328696,3.18
2,united-kingdom,english-premier-league,2007-2008,2007-08-11,derby,portsmouth,2.0,2.0,6.0,14.0,0.0,12.0,5.0,1.0,1.0,6.0,17.0,0.0,12.0,6.0,1.0,2.0,2.85,3.19,2.38,2.80,3.25,2.40,3.20,3.20,2.35,3.25,3.30,2.55,draw,draw,draw,2.0,2.0,2.0,2.0,0.323531,0.289048,0.289048,0.387421,3.19
3,united-kingdom,english-premier-league,2007-2008,2007-08-11,everton,wigan-athletic,2.0,1.0,6.0,8.0,0.0,12.0,8.0,1.0,0.0,2.0,13.0,0.0,14.0,4.0,0.0,0.0,1.66,3.45,5.25,1.66,3.40,5.50,1.73,3.50,5.40,1.75,3.65,6.00,hwin,win,loss,2.0,1.0,1.0,2.0,0.556375,0.267705,0.267705,0.175920,3.45
4,united-kingdom,english-premier-league,2007-2008,2007-08-11,middlesbrough,blackburn-rovers,1.0,2.0,13.0,16.0,0.0,10.0,6.0,1.0,3.0,3.0,16.0,0.0,4.0,4.0,0.0,4.0,2.41,3.18,2.81,2.37,3.25,2.87,2.57,3.20,2.95,2.70,3.25,2.95,awin,loss,win,1.0,2.0,2.0,1.0,0.382334,0.289756,0.289756,0.327909,3.18


In [8]:
season_dfs[-1].head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean
0,united-kingdom,english-premier-league,2017-2018,2017-08-11,arsenal,leicester-city,4.0,3.0,9.0,9.0,0.0,27.0,10.0,2.0,0.0,4.0,12.0,0.0,6.0,3.0,2.0,1.0,1.51,4.43,6.44,1.53,4.5,6.50,1.38,5.19,8.92,1.55,4.60,6.89,hwin,win,loss,4.0,3.0,3.0,4.0,0.634788,0.216372,0.216372,0.148840,4.43
1,united-kingdom,english-premier-league,2017-2018,2017-08-12,brighton-and-hove-albion,manchester-city,0.0,2.0,3.0,6.0,0.0,6.0,2.0,0.0,0.0,10.0,9.0,0.0,14.0,4.0,0.0,2.0,10.10,5.25,1.32,11.00,5.5,1.33,8.09,4.98,1.42,11.50,5.60,1.36,awin,loss,win,0.0,2.0,2.0,0.0,0.094560,0.181915,0.181915,0.723525,5.25
2,united-kingdom,english-premier-league,2017-2018,2017-08-12,chelsea,burnley,2.0,3.0,8.0,16.0,2.0,19.0,6.0,0.0,3.0,5.0,11.0,0.0,10.0,5.0,3.0,3.0,1.24,6.06,13.67,1.25,6.5,15.00,1.21,7.04,15.66,1.27,6.55,15.50,awin,loss,win,2.0,3.0,3.0,2.0,0.772004,0.157968,0.157968,0.070028,6.06
3,united-kingdom,english-premier-league,2017-2018,2017-08-12,crystal-palace,huddersfield-town,0.0,3.0,12.0,7.0,0.0,14.0,4.0,0.0,1.0,9.0,19.0,0.0,8.0,6.0,2.0,3.0,1.81,3.50,4.82,1.83,3.6,5.00,1.82,3.61,4.91,1.86,3.65,5.11,awin,loss,win,0.0,3.0,3.0,0.0,0.528356,0.273236,0.273236,0.198408,3.50
4,united-kingdom,english-premier-league,2017-2018,2017-08-12,everton,stoke-city,1.0,0.0,6.0,13.0,0.0,9.0,4.0,1.0,1.0,7.0,10.0,0.0,9.0,1.0,0.0,1.0,1.69,3.69,5.50,1.70,3.8,5.75,1.74,3.96,4.94,1.71,3.85,6.00,hwin,win,loss,1.0,0.0,0.0,1.0,0.566486,0.259448,0.259448,0.174066,3.69


In [9]:
features = ['ftGoals', 'shots', 'shotsOnTarget', 'oddsWinMean' , 'oddsDrawMean']

dfts = []
for df in season_dfs:
    dft = transform_ts_to_supervised(df, features)
    dfts.append(dft)
dft.head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [10]:
dft.tail()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [11]:
fps = []
for dft in dfts:
    fn = dft['season'].unique()[0] + '.csv'
    fp = TRANS_DATA_DIR / fn
    # print(fp)
    dft.to_csv(fp, index=None)
    fps.append(fp)

In [12]:
sub_dfs = []
for fp in fps:
    sub_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
    sub_df.sort_values(by=['date'], inplace=True)
    sub_dfs.append(sub_df)
    
full_df = pd.concat(sub_dfs, axis=0)

In [13]:
full_df.head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [14]:
full_df.tail()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [15]:
fp = CONSOL_DATA_DIR / 'epl.csv'
full_df.to_csv(fp, index=None)

In [16]:
df = pd.read_csv(fp, index_col=None)
df.head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [17]:
df.tail()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [18]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 426 columns):
nation                   4180 non-null object
league                   4180 non-null object
season                   4180 non-null object
date                     4180 non-null object
h                        4180 non-null object
a                        4180 non-null object
h_ftGoals                4180 non-null float64
a_ftGoals                4180 non-null float64
h_corners                4180 non-null float64
h_fouls                  4180 non-null float64
h_redCards               4180 non-null float64
h_shots                  4180 non-null float64
h_shotsOnTarget          4180 non-null float64
h_htGoals                4180 non-null float64
h_yellowCards            4180 non-null float64
a_corners                4180 non-null float64
a_fouls                  4180 non-null float64
a_redCards               4180 non-null float64
a_shots                  4180 non-null float64
a_sho